In [41]:
#Code for modeling Fed-batch bioreactor

#Test values (erase before combining with entire model)

mumax = .311 #1/hr
Ks = .1 #substrate saturation constant (g/L)
Yxs = 0.35 # biomass yield / substrate yield (g/g)
Si = 20 #initial cubstrate concentration (g/L)
F = 0.1 #L/hr

#Once the mu-max value of obtained from the model, we can use it here to model the cell

import numpy as np 
from scipy.integrate import odeint 
import plotly.graph_objects as go

def growthRate(X,S):
    growthRate = (mumax/(Ks + S)*S) *X
    return growthRate

#system of ODEs
def FedBatchChange(y,t): 
    X, S, V = y
    dXdt = -F/V * X + growthRate(X,S)
    dSdt = F/V * (Si-S) - growthRate(X,S) / Yxs
    dVdt = F
    return [dXdt, dSdt, dVdt]

IC = [0.05, 10, 1]
t = np.linspace(0,50)

ChangeWithTime = odeint(FedBatchChange,IC,t)
X,S,V = ChangeWithTime.transpose()


fig = go.Figure()

fig.add_trace(go.Scatter(x=t, y=X))
fig.add_trace(go.Scatter(x=t, y=S))
#fig.add_trace(go.Scatter(x=t, y=V))

fig = go.Figure()

fig.add_trace(go.Scatter(x=t, y=X, mode='lines', name='Biomass concentration (g/L)', line=dict(color='blue'), hovertemplate='Time: %{x} hr<br>Biomass concentration: %{y} g/L'))
fig.add_trace(go.Scatter(x=t, y=S, mode='lines', name='Substrate concentration (g/L)', line=dict(color='red'), hovertemplate='Time: %{x} hr<br>Substrate concentration: %{y} g/L'))
fig.add_trace(go.Scatter(x=t, y=V, mode='lines', name='Volume (L)', line=dict(color='green'), hovertemplate='Time: %{x} hr<br>Volume: %{y} L'))



# Annotations
fig.add_annotation(x=30, y=X[-1], text="Biomass", showarrow=False, arrowhead=1)
fig.add_annotation(x=40, y=S[-1]+0.5, text="Substrate", showarrow=False, arrowhead=1)
fig.add_annotation(x=30, y=V[-16], text="Volume", showarrow=False, arrowhead=1)

fig.update_layout(title='Fed-batch Bioreactor Model',
                   xaxis_title='Time (hr)',
                   yaxis_title='Concentration (g/L)')
fig.show()

In [45]:
import numpy as np
from scipy.integrate import odeint
import plotly.graph_objects as go

# Constants
mumax = 0.311  # 1/hr
Ks = 0.1  # Substrate saturation constant (g/L)
Yxs = 0.35  # Biomass yield / substrate yield (g/g)
Si = 20  # Initial substrate concentration (g/L)

# Growth rate calculation
def growth_rate(X, S, F):
    return (mumax / (Ks + S) * S) * X

# System of ODEs
def fed_batch_change(y, t, F):
    X, S, V = y
    growth_rate_val = growth_rate(X, S, F)
    return [
        -F/V * X + growth_rate_val,
        F/V * (Si - S) - growth_rate_val / Yxs,
        F
    ]

# Integration parameters
IC = [0.05, 10, 1]
t = np.linspace(0, 50)

# Define a range of flow rates to test
flow_rates = np.linspace(0.1, 1.0, 10)  # Adjust the range as needed

# Perform optimization
max_biomass = 0
optimal_flow_rate = 0
for F in flow_rates:
    change_with_time = odeint(fed_batch_change, IC, t, args=(F,))
    final_biomass = change_with_time[-1, 0]  # Biomass concentration at the end of the time period
    if final_biomass > max_biomass:
        max_biomass = final_biomass
        optimal_flow_rate = F

# Perform integration with optimal flow rate
change_with_time_optimal = odeint(fed_batch_change, IC, t, args=(optimal_flow_rate,))
X_optimal, S_optimal, V_optimal = change_with_time_optimal.transpose()

# Plotting
fig = go.Figure()
fig.add_trace(go.Scatter(x=t, y=X_optimal, mode='lines', name='Biomass concentration (g/L)', line=dict(color='blue'), hovertemplate='Time: %{x} hr<br>Biomass concentration: %{y} g/L'))
fig.add_trace(go.Scatter(x=t, y=S_optimal, mode='lines', name='Substrate concentration (g/L)', line=dict(color='red'), hovertemplate='Time: %{x} hr<br>Substrate concentration: %{y} g/L'))
fig.add_trace(go.Scatter(x=t, y=V_optimal, mode='lines', name='Volume (L)', line=dict(color='green'), hovertemplate='Time: %{x} hr<br>Volume: %{y} L'))

# Annotations
fig.add_annotation(x=30, y=X_optimal[-1], text="Biomass", showarrow=False, arrowhead=1)
fig.add_annotation(x=40, y=S_optimal[-1]+0.5, text="Substrate", showarrow=False, arrowhead=1)
fig.add_annotation(x=30, y=V_optimal[-16], text="Volume", showarrow=False, arrowhead=1)

# Layout
fig.update_layout(
    title='Optimized Fed-batch Bioreactor Model',
    xaxis_title='Time (hr)',
    yaxis_title='Concentration (g/L)'
)

# Show plot
fig.show()

print(f"Optimal flow rate for maximum biomass growth: {optimal_flow_rate} L/hr")

Optimal flow rate for maximum biomass growth: 1.0 L/hr
